<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Rekomendatelnye_sistemy/Kollaborativnaya_filtraciya/DZ_Kondratev_Kollaborativnaya_filtraciya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание по теме «Коллаборативная фильтрация»

# Задание

**Преподаватель:** Наталья Баданина, Иван Анисковец, Юлия Пономарева, Ярослав Сапронов, Егор Шишковец

Пакет SURPRISE:

* используйте данные MovieLens 1M,
* можно использовать любые модели из пакета,
* получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

# Основные таблицы в MovieLens

1. **movies**:
- **Описание**: Содержит информацию о фильмах.
- `movieId`: Уникальный идентификатор фильма.
- `title`: Название фильма.
- `genres`: Жанры фильма, обычно представлены в виде строки со списком жанров, разделенных символами `|` (например, "Action|Comedy").

2. **ratings**:
- **Описание**: Содержит оценки фильмов, выставленные пользователями.
- `userId`: Уникальный идентификатор пользователя.
- `movieId`: Уникальный идентификатор фильма (ссылается на таблицу `movies`).
- `rating`: Оценка (обычно от 0.5 до 5, с шагом 0.5).
- `timestamp`: Временная метка, указывающая, когда была оставлена оценка (обычно в формате Unix).


3. **tags** (не всегда присутствует):
- **Описание**: Содержит метки, оставленные пользователями на фильмы.
- `userId`: Уникальный идентификатор пользователя.
- `movieId`: Уникальный идентификатор фильма (ссылается на таблицу `movies`).
- `tag`: Текстовая метка, добавленная пользователем.
- `timestamp`: Временная метка, указывающая, когда была добавлена метка (обычно в формате Unix).

# Загружаем данные

In [ ]:
# Установка необходимых библиотек
!pip install scikit-surprise

In [ ]:
# Импортируем библиотеки
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [ ]:
# Задаем ссылки на данные MovieLens
movies_url = 'https://raw.githubusercontent.com/stefkong1982/netology.ru/Master/Rekomendatelnye_sistemy/Kollaborativnaya_filtraciya/movies.dat'
ratings_url = 'https://raw.githubusercontent.com/stefkong1982/netology.ru/Master/Rekomendatelnye_sistemy/Kollaborativnaya_filtraciya/ratings.dat'

In [ ]:
# Чтение данных о рейтингах с указанием нужной кодировки
ratings = pd.read_csv(ratings_url, sep='::', engine='python', names=['userId', 'movieId', 'rating', 'timestamp'], encoding='ISO-8859-1')
# Чтение данных о фильмах
movies = pd.read_csv(movies_url, sep='::', engine='python', names=['movieId', 'title', 'genres'], encoding='ISO-8859-1')

# Проверяем загруженные данные

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


# Объединение данных

In [ ]:
# Объединение таблиц рейтингов и фильмов по общему идентификатору movieId
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
# Удаляем пропуски, если такие имеются
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,9.788243e+08
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,9.782370e+08
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,9.782335e+08
3,1,Toy Story (1995),Animation|Children's|Comedy,9.0,5.0,9.782260e+08
4,1,Toy Story (1995),Animation|Children's|Comedy,10.0,5.0,9.782265e+08
...,...,...,...,...,...,...
1000381,3952,"Contender, The (2000)",Drama|Thriller,5812.0,4.0,9.920721e+08
1000382,3952,"Contender, The (2000)",Drama|Thriller,5831.0,3.0,9.862231e+08
1000383,3952,"Contender, The (2000)",Drama|Thriller,5837.0,4.0,1.011903e+09
1000384,3952,"Contender, The (2000)",Drama|Thriller,5927.0,1.0,9.798525e+08


# Создание DataFrame для Surprise

In [ ]:
# Формируем DataFrame для работы с пакетом surprise
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
# Указываем входные параметры для модели
reader = Reader(rating_scale=(0.5, 5.0))
# Загружаем данные в формат, подходящий для surprise
data = Dataset.load_from_df(dataset[['uid', 'iid', 'rating']], reader)

# Обучение и оценка модели

In [ ]:
# Разделение данных на тренировочный и тестовый наборы
trainset, testset = train_test_split(data, test_size=0.15, random_state=42)

In [ ]:
# Создаем объект модели SVD и обучаем ее на тренировочном наборе
algo = SVD(n_factors=30, n_epochs=50, lr_all=0.005, reg_all=0.05)
algo.fit(trainset)

In [ ]:
# Делаем предсказания на тестовом наборе
test_pred = algo.test(testset)

In [ ]:
# Оценка RMSE (средней квадратичной ошибки) модели
rmse = accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8503


In [ ]:
# Проведение кросс-валидации
cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8548  0.8543  0.8535  0.8524  0.8561  0.8542  0.0012  
MAE (testset)     0.6725  0.6724  0.6708  0.6708  0.6735  0.6720  0.0010  
Fit time          32.73   32.58   32.33   32.46   31.57   32.33   0.41    
Test time         3.67    3.62    3.53    5.90    3.90    4.12    0.90    


### Результаты работы модели

1. Итоговый RMSE:
   - RMSE = 0.8503 на тестовом наборе, что значительно ниже порога в 0,87. Это указывает на то, что  модель работает хорошо и справляется с предсказаниями на данных.

2. Кросс-валидация:
   - При проведении кросс-валидации (5 фолдов), среднее значение RMSE по всем фолдам составило около 0.8542. Все значения RMSE находятся также ниже 0.87, что подтверждает стабильность модели и ее способность делать точные предсказания на различных подмножествах данных.